In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
os.environ['PROJ_LIB'] = '/opt/conda/share/proj/'
n_cpus = os.cpu_count()
n_threads = n_cpus
os.environ['OMPI_MCA_rmaps_base_oversubscribe'] = '1'
os.environ['USE_PYGEOS'] = '0'
os.environ['PROJ_LIB'] = '/opt/conda/share/proj/'
os.environ['NUMEXPR_MAX_THREADS'] = f'{n_threads}'
os.environ['NUMEXPR_NUM_THREADS'] = f'{n_threads}'
os.environ['OMP_THREAD_LIMIT'] = f'{n_threads}'
os.environ["OPENBLAS_NUM_THREADS"] = f'{n_threads}' 
os.environ["MKL_NUM_THREADS"] = f'{n_threads}'
os.environ["VECLIB_MAXIMUM_THREADS"] = f'{n_threads}'
os.environ["OMP_DYNAMIC"] = f'TRUE'
os.environ['TREELITE_BIND_THREADS'] = '0'
os.environ["OMP_NUM_THREADS"] = f'{n_threads}'
os.environ["OPENBLAS_NUM_THREADS"] = f'{n_threads}'
os.environ["MKL_NUM_THREADS"] = f'{n_threads}'
os.environ["NUMEXPR_NUM_THREADS"] = f'{n_threads}'
import pathlib
import tl2cgen
import treelite
import joblib
import numpy as np
import skmap_bindings as sb
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
import treelite.sklearn
import warnings
import numpy as np
import time
import shutil

folder_path = '/mnt/ripley/global_soc/scikit-map/global-soil-mapping'
# prop_list = ['ocd', 'soc', 'bulk.density', 'ph.h2o' ,'coarse']
prop_list = ['ocd', 'soc', 'bulk.density', 'ph.h2o']

transforms_dict = {
    'ocd': 'log1p',
    'soc': 'log1p',
    'bulk.density': None,
    'ph.h2o': None,
    'coarse': 'log1p'
}
version = '20250204'

tmp_folder = 'tmp_compile'
os.makedirs(tmp_folder, exist_ok=True)

out_folder = f'/mnt/slurm/jobs/global_soc'
nthread = 96

# generate the production model with all the data we have

In [2]:
for prop in prop_list:
    print(f'\n{prop}--------------------------------------------------------------')
    model_rf = joblib.load(f'{folder_path}/production/model_rf.{prop}_production_v{version}.joblib')
    treelite_model = treelite.sklearn.import_model(model_rf)
    
    
    
    covs_path = f'{out_folder}/model_rf.{prop}_production_v{version}.covs'
    output_path = f'{out_folder}/model_rf.{prop}_production_v{version}.so'
    os.makedirs(tmp_folder, exist_ok=True)
    

    with open(covs_path, "w") as file:
        for item in list(model_rf.feature_names_in_):
            file.write(item + "\n")
            
    # tl2cgen.export_lib(treelite_model,
    #                    libpath = output_path,
    #                    nthread = nthread,
    #                    verbose=True,
    #                    toolchain="gcc",
    #                    # toolchain = '/opt/intel/oneapi/compiler/2024.0/bin/icx-cc -O3 -xCORE-AVX512 -qopt-zmm-usage=high',
    #                    params={
    #                         "verbose": 1,
    #                         "quantize": 1,
    #                         "parallel_comp": 96,
    #                    })

     tl2cgen.generate_c_code(treelite_model, tmp_folder, {"verbose": 1,"quantize": 1,"parallel_comp": 96}, verbose=True)
     temp_libpath = tl2cgen.create_shared('/opt/intel/oneapi/compiler/2024.0/bin/icx-cc -O3 -xCORE-AVX512 -qopt-zmm-usage=high',
                                          tmp_folder,
                                          nthread = nthread,
                                          verbose = True,
                                          long_build_time_warning = False)
     shutil.move(temp_libpath, output_path)
     shutil.rmtree(tmp_folder)
    


ocd--------------------------------------------------------------

soc--------------------------------------------------------------

bulk.density--------------------------------------------------------------

ph.h2o--------------------------------------------------------------
